In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jieba 
import multiprocessing
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split 
from sklearn.utils import shuffle 
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout,Activation
from keras.models import model_from_yaml
%matplotlib inline

D:\Program Files (x86)\anaconda\envs\reputation\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
# set parameters(word2vec)
vocab_dim = 200
maxlen = 100
n_iterations = 1  # ideally more..
n_exposures = 10
window_size = 7
cpu_count = multiprocessing.cpu_count()

In [3]:
data=pd.read_csv('data/train_first.csv')
#data=data.drop(['Id'],axis=1)
#data.head()
data=data.sample(frac=1).reset_index(drop=True) 

#划分x  y
X=data['Discuss']
Y=data['Score']
 
#y ONEHOT
lb = preprocessing.LabelBinarizer()
lb.fit([1,2,3,4,5])
Y=lb.transform(Y)

In [4]:
X[0]

'我只能感叹以前人的建筑理念太超前，总是建造出比较宏伟值得人去纪念。小的时候真的有摸过回音壁，可是现在回音壁也只能看看摸不到的状态，很喜欢天坛的建筑，总觉得又梦回清朝，感受着这偌大的北京城。'

In [5]:
#加载停用词
def get_stopwords(path):
    return [line.strip() for line in open(path,'r',encoding='utf-8').readlines()]
#句子去停用词
def removestopwords(sentence):
        stopwords_list=get_stopwords('data/stopwords.txt')
        outstr=[]
        for word in sentence:
            if not word in stopwords_list:
                if word!='\n' and word!='\t':
                     outstr.append(word)
        return outstr

In [6]:
#分词 并去掉停用词
def cut(sentence):
    return removestopwords(jieba.cut(sentence))
#分词后的word
sentences=[cut(x) for x in X]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\zhanggd\AppData\Local\Temp\5\jieba.cache
Loading model cost 2.688 seconds.
Prefix dict has been built succesfully.


In [7]:
sentences[0]

['只能',
 '感叹',
 '建筑',
 '理念',
 '太超前',
 '建造',
 '宏伟',
 '值得',
 '人去',
 '纪念',
 '真的',
 '摸',
 '回音壁',
 '回音壁',
 '只能',
 '摸不到',
 '状态',
 '喜欢',
 '天坛',
 '建筑',
 '总',
 '梦回',
 '清朝',
 '感受',
 '偌大',
 '北京城']

In [12]:
import itertools
from collections import Counter
def get_vocab(sentences):
    counts = Counter(list(itertools.chain.from_iterable(sentences)))
    #选择超过10次的value
    vocab_list=[]
    for word in counts:
        if counts[word]>=10:
            vocab_list.append(word)        
    vocab = sorted(vocab_list)
    vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}
    return vocab,vocab_to_int
                    
#获取字典  字典索引表
vocab,vocab_to_int=get_vocab(sentences)

In [20]:
def get_sentence2int(sentences,vocab_to_int):
    reviews_ints = []
    for each in sentences:
        int_eachsententce=[]
        for word in each:
            if word in vocab_to_int:
                int_eachsententce.append(vocab_to_int[word])
            else:
                int_eachsententce.append(0)
        reviews_ints.append(int_eachsententce)
    reviews_ints=sequence.pad_sequences(reviews_ints, maxlen=maxlen)
    return reviews_ints

In [21]:
X=get_sentence2int(sentences,vocab_to_int)

In [22]:
#搭建网络结构
input_x=tf.placeholder(shape=[None,None],dtype=tf.int32,name='input_x')
label=tf.placeholder(shape=[None,None],dtype=tf.int32,name='target')
lr=tf.placeholder(dtype=tf.float32,name='learning_rate')
#sequence_lengths=tf.placeholder(shape=[None],dtype=tf.int32,name='sequence_lengths')
keep_prob=tf.placeholder(dtype=tf.float32,name='keep_prob')
#embedding_placeholder = tf.placeholder(tf.float32, [None, None],name='embedding_placeholder')
#super params
hidden_dim=128
batch_size=100
layer_num=2
#bilstm
# with tf.variable_scope('bi_lstm'):
#     #lstm层
#     lstm_fw_cell=rnn.BasicLSTMCell(hidden_dim,forget_bias=1.0,state_is_tuple=True)
#     lstm_bw_cell=rnn.BasicLSTMCell(hidden_dim,forget_bias=1.0,state_is_tuple=True)
#     #dropout
#     lstm_fw_cell=rnn.DropoutWrapper(cell=lstm_fw_cell,input_keep_prob=1.0,out_keep_prob=keep_prob)
#     lstm_bw_cell=rnn.DropoutWrapper(cell=lstm_bw_cell,input_keep_prob=1.0,out_keep_prob=keep_prob)
#     #多层lstm
#     cell_fw=rnn.MultiRNNCell([lstm_fw_cell]*layer_num, state_is_tuple=True)
#     cell_bw=rnn.MultiRNNCell([lstm_fw_cell]*layer_num, state_is_tuple=True)
#     #初始状态
#     initial_state_fw=cell_fw.zero_state(batch_size,tf.float32)
#     initial_state_bw=cell_bw.zero_state(batch_size,tf.float32)


#with tf.variable_scope('embedding'):
#embedding = tf.Variable(tf.constant(0.0, shape=[len(index_dict)+1,vocab_dim]),trainable=True, name="embedding")
embedding = tf.Variable(tf.random_uniform((len(vocab)+1,vocab_dim), -1, 1),name='embedding')
#tf.assign(embedding,embedding_placeholder)
#print(embedding)
embed = tf.nn.embedding_lookup(embedding, input_x)
#lstm
#with tf.variable_scope('lstm'):

def get_lstm_dropout():
    lstm_cell=tf.contrib.rnn.BasicLSTMCell(hidden_dim)
    return tf.contrib.rnn.DropoutWrapper(cell=lstm_cell,output_keep_prob=keep_prob)
cell=tf.contrib.rnn.MultiRNNCell([get_lstm_dropout() for _ in range(layer_num)] )
initial_state=cell.zero_state(tf.shape(input_x)[0],tf.float32)
outputs, final_state=tf.nn.dynamic_rnn(cell,embed,initial_state=initial_state)

#with tf.variable_scope('optmizer'):
predictions=tf.contrib.layers.fully_connected(outputs[:,-1],5,activation_fn=tf.nn.softmax)
cost=tf.losses.mean_squared_error(label,predictions)
optimizer=tf.train.AdamOptimizer(lr).minimize(cost)
#validation accuracy
correct_pred=tf.equal(tf.cast(tf.round(predictions),tf.int32),label)
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [23]:
#batching
def get_batching(x,y,batch_size=100):
    n_batches=len(x)//batch_size
    x,y=x[:n_batches*batch_size],y[:n_batches*batch_size]
    for ii in range(0,len(x),batch_size):
        yield x[ii:ii+batch_size],y[ii:ii+batch_size]

In [24]:
#训练
#super params
epochs=10
saver=tf.train.Saver()
with tf.Session()as sess:
    writer = tf.summary.FileWriter("logs/", sess.graph)
    sess.run(tf.global_variables_initializer())
    x_train, x_test, y_train, y_test=train_test_split(X, Y, test_size=0.2)#get_data(index_dict,word_vectors,combined,Y)
    iteration = 1
    for e in range(epochs):
        for ii, (x, y) in enumerate(get_batching(x_train, y_train, batch_size), 1):
            feed={input_x:x,label:y,lr:0.001,keep_prob:0.5}
            loss,_=sess.run([cost,optimizer],feed_dict=feed)
            tf.summary.scalar('loss',loss)
            if iteration%50==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))
            if iteration%100==0:
                val_acc = []
                for x, y in get_batching(x_test, y_test, batch_size):
                    feed = {input_x: x,
                            label: y,                            
                            keep_prob: 1}
                    batch_acc= sess.run(accuracy, feed_dict=feed)
                    val_acc.append(batch_acc)
                tf.summary.scalar('accuracy',np.mean(val_acc))
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

Epoch: 0/10 Iteration: 50 Train loss: 0.106
Epoch: 0/10 Iteration: 100 Train loss: 0.112
Val acc: 0.841
Epoch: 0/10 Iteration: 150 Train loss: 0.099
Epoch: 0/10 Iteration: 200 Train loss: 0.104
Val acc: 0.845
Epoch: 0/10 Iteration: 250 Train loss: 0.089
Epoch: 0/10 Iteration: 300 Train loss: 0.103
Val acc: 0.848
Epoch: 0/10 Iteration: 350 Train loss: 0.100
Epoch: 0/10 Iteration: 400 Train loss: 0.102
Val acc: 0.850
Epoch: 0/10 Iteration: 450 Train loss: 0.111
Epoch: 0/10 Iteration: 500 Train loss: 0.106
Val acc: 0.852
Epoch: 0/10 Iteration: 550 Train loss: 0.075
Epoch: 0/10 Iteration: 600 Train loss: 0.101
Val acc: 0.853
Epoch: 0/10 Iteration: 650 Train loss: 0.089
Epoch: 0/10 Iteration: 700 Train loss: 0.111
Val acc: 0.854
Epoch: 0/10 Iteration: 750 Train loss: 0.098
Epoch: 0/10 Iteration: 800 Train loss: 0.097
Val acc: 0.855
Epoch: 1/10 Iteration: 850 Train loss: 0.097
Epoch: 1/10 Iteration: 900 Train loss: 0.102
Val acc: 0.856
Epoch: 1/10 Iteration: 950 Train loss: 0.101
Epoch: 1/10

Val acc: 0.848
Epoch: 9/10 Iteration: 7750 Train loss: 0.040
Epoch: 9/10 Iteration: 7800 Train loss: 0.052
Val acc: 0.851
Epoch: 9/10 Iteration: 7850 Train loss: 0.045
Epoch: 9/10 Iteration: 7900 Train loss: 0.051
Val acc: 0.851
Epoch: 9/10 Iteration: 7950 Train loss: 0.057
Epoch: 9/10 Iteration: 8000 Train loss: 0.049
Val acc: 0.850


In [25]:
predict_data=pd.read_csv('data/predict_first.csv')
predict_sententces=[cut(x) for x in predict_data['Discuss']]
predict_X=get_sentence2int(predict_sententces,vocab_to_int)

In [26]:
saver=tf.train.Saver()
model_file=tf.train.latest_checkpoint('checkpoints/')
with tf.Session()as sess:
    #sess.run(tf.global_variables_initializer()) 
    saver.restore(sess,model_file)
    predict_arr=[]
    for ii in range(0,len(predict_X),100):
        for p in sess.run(predictions,feed_dict={input_x:predict_X[ii:ii+100],keep_prob:1.0}):
            predict_arr.append(np.argmax(p)+1)

INFO:tensorflow:Restoring parameters from checkpoints/sentiment.ckpt
